In [3]:
import geopandas as gpd
import pandas as pd
import glob

/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


## Placas de sinalização

In [7]:
signs = []

for file in glob.glob('resultados/downloaded/signs/*'):
    signs.append(gpd.read_file(file))

In [8]:
df_signs = pd.concat(signs)

In [11]:
gdf_signs = gpd.GeoDataFrame(df_signs, geometry=df_signs.geometry, crs=4326)

In [13]:
gdf_signs.to_crs(epsg=31983, inplace=True)

In [15]:
gdf_signs.to_file('resultados/00-sinalizacao-sp-mapillary.gpkg', driver='GPKG')

/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [26]:
signs_grouped = gdf_signs[['value', 'id']].groupby('value').count()
signs_grouped.sort_values('id', ascending=False).head(20)

,id
value,
information--general-directions--g1,68982
regulatory--no-parking--g5,42334
regulatory--turn-right--g2,11929
regulatory--turn-left--g3,11857
regulatory--maximum-speed-limit-50--g1,10580
information--parking--g3,9781
regulatory--no-stopping--g2,6833
information--highway-interchange--g1,6780
regulatory--maximum-speed-limit-30--g1,4559


In [27]:
len(gdf_signs)

239785

In [38]:
gdf_signs.value.str.contains('no-parking', regex=False).value_counts()

False    197132
True      42535
Name: value, dtype: int64

In [39]:
gdf_signs.value.str.contains('parking', regex=False).value_counts()

False    187273
True      52394
Name: value, dtype: int64

## Área de Cobertura

In [10]:
computs = []

for file in glob.glob('resultados/downloaded/computed/*'):
    comput = gpd.read_file(file)
    computs.append(comput[comput.geometry.apply(lambda x: x.type != 'Point')])

In [11]:
df_computs = pd.concat(computs)

In [12]:
gdf_computs = gpd.GeoDataFrame(df_computs, geometry=df_computs.geometry, crs=4326)

In [14]:
gdf_computs.to_crs(epsg=31983, inplace=True)

In [15]:
gdf_computs.to_file('resultados/00-sequencias-computadas-sp-mapillary.gpkg', driver='GPKG')

/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [4]:
gdf_computs = gpd.read_file('resultados/00-sequencias-computadas-sp-mapillary.gpkg')

In [5]:
gdf_dist = gpd.read_file('zip://downloads/SIRGAS_SHP_distrito.zip!SIRGAS_SHP_distrito')

In [36]:
pvias = []

for i, dist in gdf_dist.iterrows():
    comput = gdf_computs.clip(dist.geometry)

    comput.to_file(f'resultados/computed/{dist.ds_codigo}-{dist.ds_nome}-coverage-computed.gpkg', driver='GPKG')
    
    pvia = gpd.read_file(f'../poligono-de-vias/resultado/pre-processamentos/{dist.ds_codigo}_poligono_de_vias_logradouros_{dist.ds_nome.lower().replace(" ", "_")}.gpkg')
    
    compute_count = pvia.sjoin(comput, op='intersects', how='left').reset_index()[['index', 'id']].groupby('index').count()
    
    pvia = pvia.merge(compute_count.rename(columns = {'id': 'quantidade-de-visitas'}), left_index=True, right_index=True)

    pvia.to_file(f'resultados/computed/{dist.ds_codigo}-{dist.ds_nome}-poligono-de-vias-coverage.gpkg', driver='GPKG')

    pvias.append(pvia)
    
    print(dist.ds_nome)
    # break

/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


MOEMA


/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


PARQUE DO CARMO


/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


...

JARAGUA


/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


JARDIM HELENA


/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


JARDIM PAULISTA


In [37]:
df_pvias = pd.concat(pvias)

In [38]:
gdf_pvias = gpd.GeoDataFrame(df_pvias, geometry=df_pvias.geometry)

In [41]:
gdf_pvias[['comprimento', 'cvc_classe', 'quantidade-de-visitas']]

,comprimento,cvc_classe,quantidade-de-visitas
0,325.359958,COLETORA,5
1,175.522013,ARTERIAL,8
2,225.950012,LOCAL,5
3,97.896814,LOCAL,2
4,59.835740,COLETORA,8
...,...,...,...
978,0.000000,None,21
979,0.000000,None,6
980,31.122077,ARTERIAL,28
981,0.000000,None,2


In [45]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [46]:
gdf_pvias[['comprimento', 'cvc_classe', 'quantidade-de-visitas']].groupby('cvc_classe').agg(
    {
        'cvc_classe': 'count',
        'comprimento': 'sum',
        'quantidade-de-visitas': 'mean'
    }
)

,cvc_classe,comprimento,quantidade-de-visitas
cvc_classe,,,
ARTERIAL,15084,1238590.30744,10.35388
COLETORA,39953,2759852.74889,3.65492
LOCAL,103543,8136525.80673,1.01293
RODOVIA,303,97935.66585,26.86139
VIA DE PEDESTRES,60,6292.15751,3.03333
VTR,243,83700.16625,53.26337


In [49]:
gdf_pvias[gdf_pvias['quantidade-de-visitas'] == 0]

,contagem,lg_codlog,lg_seg_id,comprimento,cvc_classe,geometry,quantidade-de-visitas
46,1,None,NaN,0.00000,None,"POLYGON ((331515.746 7391853.257, 331534.622 7...",0
68,1,117080,176132.00000,82.70980,LOCAL,"POLYGON ((331434.616 7391908.775, 331434.705 7...",0
72,1,434426,175363.00000,116.09828,LOCAL,"POLYGON ((331513.680 7391844.608, 331513.104 7...",0
84,1,None,NaN,0.00000,None,"POLYGON ((331494.988 7389153.619, 331492.808 7...",0
91,1,008826,176675.00000,84.22629,LOCAL,"POLYGON ((331424.636 7391909.454, 331451.803 7...",0
...,...,...,...,...,...,...,...
859,1,None,NaN,0.00000,None,"POLYGON ((329045.762 7392352.623, 329045.267 7...",0
925,1,None,NaN,0.00000,None,"POLYGON ((328779.952 7394450.755, 328780.347 7...",0
958,1,None,NaN,0.00000,None,"POLYGON ((328612.050 7394204.200, 328614.102 7...",0
963,1,394858,273406.00000,158.11471,LOCAL,"POLYGON ((328578.711 7393344.246, 328563.655 7...",0


In [51]:
gdf_pvias[gdf_pvias['quantidade-de-visitas'] == 0][['comprimento', 'cvc_classe', 'quantidade-de-visitas']].groupby('cvc_classe').agg(
    {
        'cvc_classe': 'count',
        'comprimento': 'sum',
        'quantidade-de-visitas': 'mean'
    }
)


,cvc_classe,comprimento,quantidade-de-visitas
cvc_classe,,,
ARTERIAL,1189,79638.91474,0.00000
COLETORA,16287,1039911.21827,0.00000
LOCAL,77614,5808016.19216,0.00000
RODOVIA,7,1081.74393,0.00000
VIA DE PEDESTRES,24,1819.43181,0.00000
VTR,14,698.50642,0.00000
